In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Senior Project/

/content/drive/MyDrive/Senior Project


In [ ]:
!unzip -q utkface.zip -d data


replace data/UTKFace/20_1_0_20170109212842300.jpg.chip.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/UTKFace/26_0_0_20170117172630031.jpg.chip.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
from google.colab import files
import tensorflow as tf
import numpy as npy

import matplotlib.pyplot as plt
import os
import datetime

In [ ]:

MODEL_INPUT_IMAGE_SIZE = [ 200 , 200 ]

TRAIN_TEST_SPLIT = 0.3

N = 20000

def parse_image( filename ):

    image_raw = tf.io.read_file( filename )
    image = tf.image.decode_jpeg( image_raw , channels=3 ) 
    image = tf.image.resize( image , MODEL_INPUT_IMAGE_SIZE ) / 255

    parts = tf.strings.split( tf.strings.split( filename , '/' )[ 2 ] , '_' )

    age = tf.strings.to_number( parts[ 0 ] ) / 116

    return image , age

list_ds = tf.data.Dataset.list_files( 'data/UTKFace/*' , shuffle=True )

dataset = list_ds.map( parse_image , num_parallel_calls=tf.data.AUTOTUNE )
dataset = dataset.take( N )


In [ ]:

num_examples_in_test_ds = int( dataset.cardinality().numpy() * TRAIN_TEST_SPLIT )

test_ds = dataset.take( num_examples_in_test_ds )
train_ds = dataset.skip( num_examples_in_test_ds )

print( 'Num examples in train ds {}'.format( train_ds.cardinality() ) )
print( 'Num examples in test ds {}'.format( test_ds.cardinality() ) )


Num examples in train ds 14000
Num examples in test ds 6000


In [ ]:

leaky_relu_alpha = 0.2

lite_model = False

def conv( x , num_filters , kernel_size=( 3 , 3 ) , strides=1 ):
    if lite_model:
        x = tf.keras.layers.SeparableConv2D( num_filters ,
                                            kernel_size=kernel_size ,
                                            strides=strides, 
                                            use_bias=False ,
                                            kernel_initializer=tf.keras.initializers.HeNormal() ,
                                            kernel_regularizer=tf.keras.regularizers.L2( 1e-5 )
                                             )( x )
    else:
        x = tf.keras.layers.Conv2D( num_filters ,
                                   kernel_size=kernel_size ,
                                   strides=strides ,
                                   use_bias=False ,
                                   kernel_initializer=tf.keras.initializers.HeNormal() ,
                                   kernel_regularizer=tf.keras.regularizers.L2( 1e-5 )
                                    )( x )

    x = tf.keras.layers.BatchNormalization()( x )
    x = tf.keras.layers.LeakyReLU( leaky_relu_alpha )( x )
    return x

def dense( x , filters , dropout_rate ):
    x = tf.keras.layers.Dense( filters , kernel_regularizer=tf.keras.regularizers.L2( 0.1 ) , bias_regularizer=tf.keras.regularizers.L2( 0.1 ) )( x )
    x = tf.keras.layers.LeakyReLU( alpha=leaky_relu_alpha )( x )
    x = tf.keras.layers.Dropout( dropout_rate )( x )
    return x


num_blocks = 6
num_filters = [ 16 , 32 , 64 , 128 , 256 , 256 ]
kernel_sizes = [ 3 , 3 , 3 , 3 , 3 , 3 ]

inputs = tf.keras.layers.Input( shape=MODEL_INPUT_IMAGE_SIZE + [ 3 ] )

x = inputs
for i in range( num_blocks ):
    x = conv( x , num_filters=num_filters[ i ] , kernel_size=kernel_sizes[ i ] )
    x = tf.keras.layers.MaxPooling2D()( x )

x = tf.keras.layers.Flatten()( x )
conv_output = x 

x = dense( conv_output , 256 , 0.6 )
x = dense( x , 64 , 0.4 )
x = dense( x , 32 , 0.2 )
outputs = tf.keras.layers.Dense( 1 , activation='relu' )( x )

model = tf.keras.models.Model( inputs , outputs )

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 198, 198, 16)      432       
                                                                 
 batch_normalization (BatchN  (None, 198, 198, 16)     64        
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 198, 198, 16)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 32)        4608  

In [ ]:

learning_rate = 0.001

num_epochs = 15
batch_size = 128 

train_ds = train_ds.batch( batch_size )
test_ds = test_ds.batch( batch_size )

save_dir_ = 'model_1' 
save_dir = save_dir_ + '/{epoch:02d}-{val_mae:.2f}.h5'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( 
    save_dir , 
    save_best_only=True , 
    monitor='val_mae' , 
    mode='min' , 
)

tb_log_name = 'model_1'
logdir = os.path.join( "tb_logs" , tb_log_name )
tensorboard_callback = tf.keras.callbacks.TensorBoard( logdir )

def scheduler( epochs , learning_rate ):
    if epochs < num_epochs * 0.25:
        return learning_rate
    elif epochs < num_epochs * 0.5:
        return 0.0005
    elif epochs < num_epochs * 0.75:
        return 0.0001
    else:
        return 0.000095

lr_schedule_callback = tf.keras.callbacks.LearningRateScheduler( scheduler )

early_stopping_callback = tf.keras.callbacks.EarlyStopping( monitor='val_mae' , patience=10 )

model.compile( 
    loss=tf.keras.losses.mean_absolute_error ,
    optimizer = tf.keras.optimizers.Adam( learning_rate ) , 
    metrics=[ 'mae' ]
)


In [ ]:

model.fit( 
    train_ds, 
    epochs=num_epochs,  
    validation_data=test_ds, 
    callbacks=[ checkpoint_callback , tensorboard_callback , lr_schedule_callback , early_stopping_callback ]
)


Epoch 1/15


KeyboardInterrupt: ignored

In [ ]:
while True:pass

KeyboardInterrupt: ignored

In [ ]:

p = model.evaluate( test_ds )  
print( p )


47/47 [==============================] - 93s 2s/step - loss: 0.0892 - mae: 0.0568
[0.08918748050928116, 0.05675911903381348]


In [ ]:

converter = tf.lite.TFLiteConverter.from_keras_model( model )
converter.optimizations = [ tf.lite.Optimize.DEFAULT ]
converter.target_spec.supported_types = [ tf.float16 ]
buffer = converter.convert()
print(model)
open( '{}_q.tflite'.format( model_name ) , 'wb' ).write( buffer )
files.download( '{}_q.tflite'.format( model_name ) )

INFO:tensorflow:Assets written to: /tmp/tmpm10o8p26/assets


INFO:tensorflow:Assets written to: /tmp/tmpm10o8p26/assets


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

batch_size = 128

model = tf.keras.models.load_model( 'model_lite_age.h5' )
p = model.evaluate( test_ds.batch( batch_size ) )
print( p )


OSError: ignored

In [ ]:

model_name = 'model_age' 
model_name_ = model_name + '.h5'

model.save( model_name_ )
files.download( model_name_ ) 


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

converter = tf.lite.TFLiteConverter.from_keras_model( model )
buffer = converter.convert()

open( '{}_nonq.tflite'.format( model_name ) , 'wb' ).write( buffer )
files.download( '{}_nonq.tflite'.format( model_name ) )
